In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# https://www.kaggle.com/remekkinas/lstm-seq2seq-encoder-decoder#SIMPLE-NN-MODEL
import pandas as pd
import numpy as np
from numpy import array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Conv1D, MaxPooling1D, Flatten, ConvLSTM2D, Dropout
import tensorflow.keras.backend as K

import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm

In [3]:
n_steps = 8 # we use 12h window
n_lookup = 1 # predict series of 4 values in time t1, t2, t3, t4

In [4]:
# df_train = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv")
# df_test = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv")
# df_sub = pd.read_csv("../input/tabular-playground-series-jul-2021/sample_submission.csv")

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_sub = pd.read_csv("sample_submission.csv")

print(df_test.shape)
print(df_sub.shape)

features = ['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']
targets = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']
targets_values = np.log1p(df_train[targets]).values


df_test = pd.concat([df_train[len(df_train)-n_steps-1:len(df_train)-1].drop(targets , axis = 1), df_test])

df_all = pd.concat([df_train.drop(targets , axis = 1), df_test])

df_all['date_time'] = pd.to_datetime(df_all['date_time'])


df_train.set_index('date_time', inplace=True)
df_test.set_index('date_time', inplace=True)
print(df_test.shape)
print(df_all.shape)

(2247, 9)
(2247, 4)
(2255, 8)
(9366, 9)


In [5]:
def cycle_sin_cos_coder(data, cols):
    for col in cols:
        data[col + '_s'] = np.sin(2 * np.pi * data[col]/data[col].max())
        data[col + '_c'] = np.cos(2 * np.pi * data[col]/data[col].max())
    return data

In [6]:
df_all['month'] = df_all['date_time'].dt.month
df_all['day'] = df_all['date_time'].dt.day
df_all['hour'] = df_all['date_time'].dt.hour

df_all = cycle_sin_cos_coder(df_all, ['month','day','hour'])
df_all.drop(['month','day','hour'], axis=1, inplace=True)
df_all.set_index('date_time', inplace=True)

print(df_all.shape)

(9366, 14)


In [7]:
df_train = df_all[:len(df_train)]

df_train [targets] = targets_values
df_test = df_all[len(df_train):]

In [8]:
train, test = train_test_split(df_train, shuffle = False, train_size=0.8)

In [9]:
for i in train[features].columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    
    s_train = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_test = scaler.transform(test[i].values.reshape(-1,1))
    s_df_test = scaler.transform(df_test[i].values.reshape(-1,1))
    
    s_train = np.reshape(s_train,len(s_train))
    s_test = np.reshape(s_test,len(s_test))
    s_df_test = np.reshape(s_df_test,len(s_df_test))

    train[i] = s_train
    test[i] = s_test
    df_test[i] = s_df_test

In [10]:
def split_sequences(Xsequences, ysequences, n_steps = 6, n_out = 1):
    X, y = list(), list()

    for i in range(len(Xsequences)):
        end_index = i + n_steps
        out_end_index = end_index + n_out
        
        if out_end_index > len(Xsequences):
            break
        
        seq_x = Xsequences.iloc[i : end_index, :] 
        if isinstance(ysequences, pd.core.series.Series):
            seq_y = ysequences.iloc[end_index : out_end_index]
            y.append(seq_y)

        X.append(seq_x)
        
    return array(X), array(y)

In [11]:
Xtrain_seq_tcm, ytrain_seq_tcm = split_sequences(train.drop(targets, axis = 1), train['target_carbon_monoxide'], n_steps, n_lookup)
Xtest_seq_tcm, ytest_seq_tcm = split_sequences(test.drop(targets, axis = 1), test['target_carbon_monoxide'], n_steps, n_lookup)

Xtrain_seq_tb, ytrain_seq_tb = split_sequences(train.drop(targets, axis = 1), train['target_benzene'], n_steps, n_lookup)
Xtest_seq_tb, ytest_seq_tb = split_sequences(test.drop(targets, axis = 1), test['target_benzene'], n_steps, n_lookup)

Xtrain_seq_tno, ytrain_seq_tno = split_sequences(train.drop(targets, axis = 1), train['target_nitrogen_oxides'], n_steps, n_lookup)
Xtest_seq_tno, ytest_seq_tno = split_sequences(test.drop(targets, axis = 1), test['target_nitrogen_oxides'], n_steps, n_lookup)

n_features = Xtrain_seq_tcm.shape[2]

print(Xtrain_seq_tcm.shape, ytrain_seq_tcm.shape)
print(Xtest_seq_tcm.shape, ytest_seq_tcm.shape)

(5680, 8, 14) (5680, 1)
(1415, 8, 14) (1415, 1)


In [12]:
np.set_printoptions(suppress=True, linewidth=255)

Xtest_sub, _ = split_sequences(df_test, [], n_steps, n_lookup)
print(Xtest_sub[0])
print(Xtest_sub.shape)

[[-0.66831683 -0.58092848 -0.79686725 -0.61577008 -0.52286481 -0.38872892 -0.64293278 -0.56189824 -0.          1.         -0.          1.         -0.94226092 -0.33487961]
 [-0.68316832 -0.4880803  -0.80053534 -0.50626382 -0.35242373 -0.39942411 -0.68367995 -0.4384013  -0.          1.         -0.          1.         -0.99766877 -0.06824241]
 [-0.77722772 -0.42032622 -0.80410429 -0.62092852 -0.45511261 -0.37345008 -0.72620611 -0.51893534 -0.          1.         -0.          1.         -0.97908409  0.20345601]
 [-0.83168317 -0.36260979 -0.81937147 -0.56904937 -0.52847963 -0.23970147 -0.69316786 -0.47939578 -0.          1.         -0.          1.         -0.88788522  0.46006504]
 [-0.82673267 -0.42032622 -0.82799643 -0.50626382 -0.44201135 -0.35405771 -0.69833538 -0.44131114 -0.          1.         -0.          1.         -0.73083596  0.68255314]
 [-0.83168317 -0.3902133  -0.83840587 -0.47354458 -0.46846341 -0.2616795  -0.69503156 -0.44738756 -0.          1.         -0.          1.        

In [13]:
np.set_printoptions(suppress=True, linewidth=255)

num_seq_show = 3

for i in range(num_seq_show):
    print(f'X{i}\n {Xtrain_seq_tcm[i]}')
    print(f'y{i}\n {ytrain_seq_tcm[i]} \n\n')

X0
 [[-0.63366337 -0.06900878 -0.4604937   0.06366986 -0.09688689 -0.13897867  0.00800542 -0.11361205  1.          0.          0.89780454 -0.44039415 -0.97908409  0.20345601]
 [-0.62871287 -0.08657465 -0.49251512 -0.09565217 -0.34593549  0.02732562 -0.24012029 -0.35538534  1.          0.          0.89780454 -0.44039415 -0.88788522  0.46006504]
 [-0.65841584  0.18695107 -0.46802816 -0.01783346 -0.29427912 -0.13404243 -0.12474057 -0.26458128  1.          0.          0.89780454 -0.44039415 -0.73083596  0.68255314]
 [-0.73762376  0.3425345  -0.43184297 -0.03389831 -0.29502776 -0.08385732 -0.16675844 -0.13945826  1.          0.          0.89780454 -0.44039415 -0.51958395  0.8544194 ]
 [-0.69306931  0.25721455 -0.42976108 -0.10611643 -0.39023021  0.00781571 -0.26926172 -0.2515726   1.          0.          0.89780454 -0.44039415 -0.26979677  0.96291729]
 [-0.72772277  0.20200753 -0.43372658 -0.18142962 -0.58387922  0.28553799 -0.22936168 -0.4083615   1.          0.          0.89780454 -0.4403

In [14]:
def rmsle(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return K.sqrt(msle(y_true, y_pred)) 

In [15]:
# A. LSTM -> Encoder-Decoder -> LSTM -> Dense¶
model_tcm = Sequential()
model_tcm.add(LSTM(100, activation='tanh', input_shape=(n_steps, n_features)))
model_tcm.add(RepeatVector(n_lookup))
model_tcm.add(LSTM(100, activation='tanh', return_sequences=True))
model_tcm.add(TimeDistributed(Dense(1)))
model_tcm.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.02), loss= rmsle)

model_tcm.summary()

2021-07-22 21:04:19.421315: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               46000     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 1, 100)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 100)            80400     
_________________________________________________________________
time_distributed (TimeDistri (None, 1, 1)              101       
Total params: 126,501
Trainable params: 126,501
Non-trainable params: 0
_________________________________________________________________


In [16]:
tf.keras.utils.plot_model(model_tcm)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [17]:
es = tf.keras.callbacks.EarlyStopping(patience=10, verbose=0, min_delta=0.001, monitor='val_loss', mode='auto', restore_best_weights=True)
red_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)

def plot_model_learning(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [18]:
N_SAMPLE = 20
yhat_tcm = np.zeros((Xtest_sub.shape[0],1))

for samples in tqdm(range(N_SAMPLE)):
    tf.keras.backend.clear_session()
    
    model_tcm.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.02), loss= rmsle)
    history_tcm = model_tcm.fit(Xtrain_seq_tcm, ytrain_seq_tcm, 
                                validation_data = (Xtest_seq_tcm, ytest_seq_tcm), 
                                epochs=100, 
                                verbose = 0,
                                batch_size = 16, 
                                callbacks=[es, red_lr])
    
  
    yhat_tcm += np.expm1(model_tcm.predict(Xtest_sub)).reshape(-1,1)

yhat_tcm = yhat_tcm / N_SAMPLE

  0%|          | 0/20 [00:00<?, ?it/s]

2021-07-22 21:04:20.026871: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


In [19]:
# B. ConvLSTM2D -> Encoder-Decoder -> LSTM -> Dense
n_sub_steps = 4
n_length = 2

model_tb = Sequential()
model_tb.add(ConvLSTM2D(64, (1,2), activation='relu', input_shape=(n_sub_steps, 1, n_length, n_features)))
model_tb.add(Flatten())
model_tb.add(RepeatVector(n_lookup))
model_tb.add(LSTM(200, activation='relu', return_sequences=True))
model_tb.add(TimeDistributed(Dense(100, activation='relu')))
model_tb.add(TimeDistributed(Dense(1)))

model_tb.compile(loss='mse', optimizer='adam')

model_tb.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 1, 1, 64)          40192     
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
repeat_vector (RepeatVector) (None, 1, 64)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 1, 200)            212000    
_________________________________________________________________
time_distributed (TimeDistri (None, 1, 100)            20100     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 1)              101       
Total params: 272,393
Trainable params: 272,393
Non-trainable params: 0
__________________________________________________

In [20]:
tf.keras.utils.plot_model(model_tb)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [21]:
Xtrain_seq_tb = Xtrain_seq_tb.reshape((Xtrain_seq_tb.shape[0], n_sub_steps, 1, n_length, n_features))
Xtest_seq_tb = Xtest_seq_tb.reshape((Xtest_seq_tb.shape[0], n_sub_steps , 1, n_length, n_features))

In [22]:
N_SAMPLE = 20
yhat_tb = np.zeros((Xtest_sub.shape[0],1))

for samples in tqdm(range(N_SAMPLE)):
    tf.keras.backend.clear_session()
    
    model_tb.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.02), loss= rmsle)
    history_tb = model_tb.fit(Xtrain_seq_tb, ytrain_seq_tb, 
                          validation_data = (Xtest_seq_tb, ytest_seq_tb), 
                          epochs=100,  
                          batch_size = 16, 
                          verbose = 0, 
                          callbacks=[es, red_lr])
  
    yhat_tb += np.expm1(model_tb.predict(Xtest_sub.reshape(Xtest_sub.shape[0], n_sub_steps, 1, n_length, n_features))).reshape(-1,1)

yhat_tb = yhat_tb / N_SAMPLE

  0%|          | 0/20 [00:00<?, ?it/s]

In [23]:
# CONV1D -> Encoder-Decoder -> LSTM -> Dense
model_tno = Sequential()
model_tno.add(Conv1D(64, 3, activation='relu', input_shape=(n_steps, n_features)))
model_tno.add(Conv1D(64, 3, activation='relu'))
model_tno.add(MaxPooling1D())
model_tno.add(Flatten())
model_tno.add(RepeatVector(n_lookup))
model_tno.add(LSTM(100, activation='relu', return_sequences=True))
model_tno.add(TimeDistributed(Dense(64, activation='relu')))
model_tno.add(TimeDistributed(Dense(1)))

model_tno.compile(loss='mse', optimizer='adam')

model_tno.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 6, 64)             2752      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4, 64)             12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 2, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
repeat_vector (RepeatVector) (None, 1, 128)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 1, 100)            91600     
_________________________________________________________________
time_distributed (TimeDistri (None, 1, 64)             6

In [24]:
tf.keras.utils.plot_model(model_tno)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [25]:
N_SAMPLE = 20
yhat_tno = np.zeros((Xtest_sub.shape[0],1))

for samples in tqdm(range(N_SAMPLE)):
    tf.keras.backend.clear_session()
    
    model_tno.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.02), loss= rmsle)
    history_tno = model_tno.fit(Xtrain_seq_tno, ytrain_seq_tno, 
                            validation_data = (Xtest_seq_tno, ytest_seq_tno), 
                            epochs=100, 
                            verbose = 1, 
                            batch_size = 16, 
                            callbacks=[es, red_lr])
    
  
    yhat_tno += np.expm1(model_tno.predict(Xtest_sub)).reshape(-1,1)

yhat_tno = yhat_tno / N_SAMPLE

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1/100
355/355 [==============================] - 7s 6ms/step - loss: 0.4096 - val_loss: 0.0940
Epoch 2/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0775 - val_loss: 0.0911
Epoch 3/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0701 - val_loss: 0.0888
Epoch 4/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0668 - val_loss: 0.0957
Epoch 5/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0656 - val_loss: 0.0850
Epoch 6/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0612 - val_loss: 0.0784
Epoch 7/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0624 - val_loss: 0.1006
Epoch 8/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0599 - val_loss: 0.1056
Epoch 9/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0572 - val_loss: 0.1165
Epoch 10/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0557 - val_lo

Epoch 14/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0424 - val_loss: 0.0853
Epoch 1/100
355/355 [==============================] - 5s 6ms/step - loss: 0.0525 - val_loss: 0.0752
Epoch 2/100
355/355 [==============================] - 1s 2ms/step - loss: 0.0502 - val_loss: 0.0785
Epoch 3/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0500 - val_loss: 0.0830
Epoch 4/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0504 - val_loss: 0.0808
Epoch 5/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0481 - val_loss: 0.0749
Epoch 6/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0464 - val_loss: 0.0999
Epoch 7/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0447 - val_loss: 0.0858
Epoch 8/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0451 - val_loss: 0.0826
Epoch 9/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0447 - val_lo

Epoch 2/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0427 - val_loss: 0.0769
Epoch 3/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0411 - val_loss: 0.0772
Epoch 4/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0406 - val_loss: 0.0789
Epoch 5/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0396 - val_loss: 0.0838
Epoch 6/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0377 - val_loss: 0.0851
Epoch 7/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0368 - val_loss: 0.0927
Epoch 8/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0360 - val_loss: 0.0935
Epoch 9/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0346 - val_loss: 0.0855
Epoch 10/100
355/355 [==============================] - 1s 2ms/step - loss: 0.0339 - val_loss: 0.0851
Epoch 11/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0340 - val_l

Epoch 17/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0249 - val_loss: 0.0870
Epoch 1/100
355/355 [==============================] - 6s 7ms/step - loss: 0.0372 - val_loss: 0.0832
Epoch 2/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0359 - val_loss: 0.0787
Epoch 3/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0343 - val_loss: 0.0917
Epoch 4/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0336 - val_loss: 0.0863
Epoch 5/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0329 - val_loss: 0.0873
Epoch 6/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0311 - val_loss: 0.0840
Epoch 7/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0298 - val_loss: 0.0839
Epoch 8/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0293 - val_loss: 0.0822
Epoch 9/100
355/355 [==============================] - 1s 3ms/step - loss: 0.0278 - val_lo

In [26]:
# Submission
df_sub['target_carbon_monoxide'] =  yhat_tcm
df_sub['target_benzene'] = yhat_tb
df_sub['target_nitrogen_oxides'] = yhat_tno

df_sub.to_csv('lstm_001.csv', index=False)

In [27]:
df_sub

,date_time,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2011-01-01 00:00:00,1.506565,4.910899,302.449928
1,2011-01-01 01:00:00,1.673048,5.506151,269.728941
2,2011-01-01 02:00:00,2.365980,8.217354,351.887906
3,2011-01-01 03:00:00,2.201139,8.506038,300.537603
4,2011-01-01 04:00:00,2.331467,8.790062,266.922013
...,...,...,...,...
2242,2011-04-04 10:00:00,2.356931,11.094714,220.149908
2243,2011-04-04 11:00:00,2.519680,11.416447,187.439294
2244,2011-04-04 12:00:00,2.321282,11.059037,169.660686
2245,2011-04-04 13:00:00,2.447847,11.228000,147.657148
